<a href="https://colab.research.google.com/github/rjpaxtondata/Deep_Learning_Charity_Data/blob/main/Starter_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [4]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import io

file = "Resources/charity_data.csv"
df = pd.read_csv(file)

df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [5]:
# #use google collab import data option
# from google.colab import files
# uploaded = files.upload()

In [6]:
# #Read the csv using the google collab pd.read
# import io
# df = pd.read_csv(io.StringIO(uploaded['charity_data.csv'].decode('utf-8')), sep = ',')
# df

In [7]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.

df = df.drop(['EIN', 'NAME'], axis=1)

In [8]:
# Determine the number of unique values in each column.
df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [9]:
# Look at APPLICATION_TYPE value counts for binning
app_type = df['APPLICATION_TYPE'].value_counts()

In [10]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = list(app_type[app_type < 500].index)

# Replace in dataframe
for app in application_types_to_replace:
    df['APPLICATION_TYPE'] = df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [11]:
# Look at CLASSIFICATION value counts for binning
class_df = df['CLASSIFICATION'].value_counts()
class_df

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C1370        1
C2600        1
C1820        1
C3700        1
C6100        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [12]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_df[class_df > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C1720        6
C2400        6
C4100        6
C1257        5
C1600        5
C2710        3
C1260        3
C0           3
C3200        2
C1256        2
C1246        2
C1234        2
C1267        2
Name: CLASSIFICATION, dtype: int64

In [13]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = list(class_df[class_df < 1883].index)

# Replace in dataframe
for cls in classifications_to_replace:
    df['CLASSIFICATION'] = df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [14]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies = pd.get_dummies(df)

In [15]:
# Split our preprocessed data into our features and target arrays
y = dummies['IS_SUCCESSFUL']
x = dummies.drop(columns=['IS_SUCCESSFUL'])
# Split the preprocessed data into a training and testing dataset
xtrain, xtest, ytrain, ytest = train_test_split(x, y, random_state=123)

In [16]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(xtrain)

# Scale the data
X_train_scaled = X_scaler.transform(xtrain)
X_test_scaled = X_scaler.transform(xtest)

## Compile, Train and Evaluate the Model

In [17]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_layers = len(x.columns)
#layer1=50
#layer2=25
nn = tf.keras.models.Sequential()


# First hidden layer (units=layer1)
nn.add(tf.keras.layers.Dense(units=50, input_dim=input_layers, activation='relu'))

# Second hidden layer (units=layer2)
nn.add(tf.keras.layers.Dense(units=25, activation='relu'))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 50)                2200      
                                                                 
 dense_1 (Dense)             (None, 25)                1275      
                                                                 
 dense_2 (Dense)             (None, 1)                 26        
                                                                 
Total params: 3,501
Trainable params: 3,501
Non-trainable params: 0
_________________________________________________________________


In [18]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [19]:
# Train the model
model_fit = nn.fit(X_train_scaled, ytrain, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 876us/step - loss: 0.5719 - accuracy: 0.7200
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5564 - accuracy: 0.7297
Epoch 3/100
804/804 [==============================] - 1s 865us/step - loss: 0.5543 - accuracy: 0.7304
Epoch 4/100
804/804 [==============================] - 1s 884us/step - loss: 0.5516 - accuracy: 0.7297
Epoch 5/100
804/804 [==============================] - 1s 863us/step - loss: 0.5510 - accuracy: 0.7302
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5505 - accuracy: 0.7309: 1s - loss: 0.5
Epoch 7/100
804/804 [==============================] - 1s 862us/step - loss: 0.5490 - accuracy: 0.7315
Epoch 8/100
804/804 [==============================] - 1s 818us/step - loss: 0.5482 - accuracy: 0.7328
Epoch 9/100
804/804 [==============================] - 1s 864us/step - loss: 0.5485 - accuracy: 0.7326
Epoch 10/100
804/804 [==============================] - 1s 1m

In [20]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled, ytest,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5565 - accuracy: 0.7276 - 222ms/epoch - 830us/step
Loss: 0.5565037727355957, Accuracy: 0.727580189704895


In [ ]:
# Procedure still running after 1.5hrs 
#Find the best hyperparameter options
# def create_model(hp):
#   nn_model = tf.keras.models.Sequential()
#   input_features = len(X_train_scaled[0])
  
#   #Allow kerastuner to decide which activation function to use in hidden layers
#   activation = hp.Choice('activation',['relu','tanh','sigmoid'])

#   # Allow kerastuner to decide number of neurons in first layer
#   nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
#         min_value=60,
#         max_value=200,
#         step=15), 
#         activation=activation, 
#         input_dim=input_features))                                    
  
#   ## Allow kerastuner to decide number of hidden layers and neurons in hidden layers
#   for i in range(hp.Int('num_layers', 1, 6)):
#         nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
#             min_value=1,
#             max_value=10,
#             step=2),
#             activation=activation))
  
#   ## Standard output layer with 1 sigmoid unit
#   nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

#   # Compile the model
#   nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
#   return nn_model



In [ ]:
# !pip install keras-tuner
# # Import the kerastuner library
# import kerastuner as kt

# tuner = kt.Hyperband(
#     create_model,
#     objective="val_accuracy",
#     max_epochs=100,
#     hyperband_iterations=2)

In [ ]:
# # Run the kerastuner search for best hyperparameters
# tuner.search(X_train_scaled,ytrain,epochs=100,validation_data=(X_test_scaled,ytest))

# Obtain accuracy higher than 75%

In [21]:
# Define the model#1 - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_layers = len(x.columns)
#layer1=50
#layer2=25
nn1 = tf.keras.models.Sequential()


# First hidden layer (units=layer1)
nn1.add(tf.keras.layers.Dense(units=250, input_dim=input_layers, activation='relu'))

# Second hidden layer (units=layer2)
nn1.add(tf.keras.layers.Dense(units=100, activation='relu'))

# third hidden layer (units=layer2)
nn1.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Output layer
nn1.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn1.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 250)               11000     
                                                                 
 dense_4 (Dense)             (None, 100)               25100     
                                                                 
 dense_5 (Dense)             (None, 50)                5050      
                                                                 
 dense_6 (Dense)             (None, 1)                 51        
                                                                 
Total params: 41,201
Trainable params: 41,201
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compile the model#1
nn1.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [23]:
# Train the model#1
model_fit = nn1.fit(X_train_scaled, ytrain, epochs=100)

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5659 - accuracy: 0.7223
Epoch 2/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5554 - accuracy: 0.7283
Epoch 3/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5522 - accuracy: 0.7304
Epoch 4/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5518 - accuracy: 0.7295
Epoch 5/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5492 - accuracy: 0.7320
Epoch 6/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5483 - accuracy: 0.7326
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5479 - accuracy: 0.7336
Epoch 8/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5475 - accuracy: 0.7339
Epoch 9/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5467 - accuracy: 0.7345
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5452 - accura

In [24]:
# Evaluate the model using the test data - model#1
model_loss, model_accuracy = nn1.evaluate(X_test_scaled, ytest,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5797 - accuracy: 0.7291 - 215ms/epoch - 801us/step
Loss: 0.579713761806488, Accuracy: 0.7290962338447571


In [32]:
#Model #2
input_layers = len(x.columns)
#layer1=50
#layer2=25
nn2 = tf.keras.models.Sequential()


# First hidden layer (units=layer1)
nn2.add(tf.keras.layers.Dense(units=500, input_dim=input_layers, activation='relu'))

# Second hidden layer (units=layer2)
nn2.add(tf.keras.layers.Dense(units=250, activation='relu'))

# third hidden layer (units=layer2)
nn2.add(tf.keras.layers.Dense(units=100, activation='relu'))

# Output layer
nn2.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn2.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)            (None, 500)               22000     
                                                                 
 dense_16 (Dense)            (None, 250)               125250    
                                                                 
 dense_17 (Dense)            (None, 100)               25100     
                                                                 
 dense_18 (Dense)            (None, 1)                 101       
                                                                 
Total params: 172,451
Trainable params: 172,451
Non-trainable params: 0
_________________________________________________________________


In [33]:
# Compile the model#3
nn2.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [34]:
# Import checkpoint dependencies
import os
from tensorflow.keras.callbacks import ModelCheckpoint

# Define the checkpoint path and filenames
os.makedirs("checkpoints/",exist_ok=True)
checkpoint_path = "checkpoints/weights.{epoch:02d}.hdf5"

In [35]:
# create a callback that save's odel's weight every 5 epochs 
cp_callback = ModelCheckpoint(
    filepath = checkpoint_path,
    verbose = 1,
    save_weights_only = True, 
    save_freq = 'epoch',
    period = 5
    )

In [36]:
# Train the model#3
model_fit = nn2.fit(X_train_scaled, ytrain, epochs=100, callbacks=[cp_callback])

Epoch 1/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5683 - accuracy: 0.7234
Epoch 2/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5556 - accuracy: 0.7303
Epoch 3/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5532 - accuracy: 0.7299
Epoch 4/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5512 - accuracy: 0.7308
Epoch 5/100
791/804 [============================>.] - ETA: 0s - loss: 0.5505 - accuracy: 0.7320
Epoch 00005: saving model to checkpoints\weights.05.hdf5
804/804 [==============================] - 2s 2ms/step - loss: 0.5504 - accuracy: 0.7319
Epoch 6/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5492 - accuracy: 0.7319: 0s - loss: 0.5501 
Epoch 7/100
804/804 [==============================] - 3s 3ms/step - loss: 0.5488 - accuracy: 0.7334
Epoch 8/100
804/804 [==============================] - 2s 3ms/step - loss: 0.5472 - accuracy: 0.7332
Epoch 9/100
804/804 [=========

In [37]:
# Evaluate the model using the test data - Model#2
model_loss, model_accuracy = nn2.evaluate(X_test_scaled, ytest,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5879 - accuracy: 0.7304 - 244ms/epoch - 910us/step
Loss: 0.5878677368164062, Accuracy: 0.7303789854049683


In [29]:
# Define the model#3 - deep neural net, i.e., the number of input features and hidden nodes for each layer.
input_layers = len(x.columns)
#layer1=50
#layer2=25
nn3 = tf.keras.models.Sequential()


# First hidden layer (units=layer1)
nn3.add(tf.keras.layers.Dense(units=100, input_dim=input_layers, activation='relu'))

# Second hidden layer (units=layer2)
nn3.add(tf.keras.layers.Dense(units=100, activation='sigmoid'))

# third hidden layer (units=layer2)
nn3.add(tf.keras.layers.Dense(units=50, activation='relu'))

# Output layer
nn3.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn3.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_11 (Dense)            (None, 100)               4400      
                                                                 
 dense_12 (Dense)            (None, 100)               10100     
                                                                 
 dense_13 (Dense)            (None, 50)                5050      
                                                                 
 dense_14 (Dense)            (None, 1)                 51        
                                                                 
Total params: 19,601
Trainable params: 19,601
Non-trainable params: 0
_________________________________________________________________


In [30]:
# Compile the model#3
nn3.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [31]:
# Train the model - Model#3
model_fit = nn3.fit(X_train_scaled, ytrain, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 903us/step - loss: 0.5708 - accuracy: 0.7186
Epoch 2/100
804/804 [==============================] - 1s 894us/step - loss: 0.5569 - accuracy: 0.7279
Epoch 3/100
804/804 [==============================] - 1s 923us/step - loss: 0.5539 - accuracy: 0.7295
Epoch 4/100
804/804 [==============================] - 1s 965us/step - loss: 0.5515 - accuracy: 0.7316
Epoch 5/100
804/804 [==============================] - 1s 934us/step - loss: 0.5499 - accuracy: 0.7286
Epoch 6/100
804/804 [==============================] - 1s 938us/step - loss: 0.5491 - accuracy: 0.7320
Epoch 7/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5485 - accuracy: 0.7308
Epoch 8/100
804/804 [==============================] - 1s 2ms/step - loss: 0.5480 - accuracy: 0.7322
Epoch 9/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5469 - accuracy: 0.7332
Epoch 10/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5

In [ ]:
# Evaluate the model using the test data - Model #3
model_loss, model_accuracy = nn3.evaluate(X_test_scaled, ytest,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5821 - accuracy: 0.7290 - 442ms/epoch - 2ms/step
Loss: 0.582116425037384, Accuracy: 0.7289795875549316


In [38]:
# Export our model to HDF5 file
nn.save("AlphabetSoupCharity_Optimization.h5")